| Field Index | Variable         | Meaning                         |
| ----------- | ---------------- | ------------------------------- |
| `f[0]`      | Transform matrix | Position and orientation        |
| `f[1]`      | Velocity vector  | Movement direction/speed        |
| `f[2]`      | Health level     | Aircraft damage level           |
| `f[3]`      | Wreck flag       | Whether the aircraft is wrecked |
| `f[4]`      | Brake level      | Brake force on wheels           |
| `f[5]`      | Flaps level      | Wing flaps for lift/drag        |
| `f[6]`      | Landed flag      | Whether the aircraft has landed |
| `f[7]`      | Target name      | Target of the targeting system  |


# Libraries

In [1]:
import numpy as np

import os

import pandas as pd

import sqlite3

# Constants

In [2]:
DB_DIR = os.path.join('..', 'data', 'external', 'database.db')
SAVE_DIR = os.path.join('..', 'data', 'interim', 'version_01.csv')
RECORD_ID = 3
COLUMN_NAMES = ['transform_matrix', 'velocity', 'health_level', 'wreck_flag', 'brake_level', 'flaps_level', 'landed_level', 'target_name']
FINAL_FEATURES = 

# Import Data

In [3]:
conn = sqlite3.connect(DB_DIR)
df_our = pd.read_sql_query(f'SELECT * FROM ally_1_machine_state WHERE id_rec = {RECORD_ID}', conn)
df_bandit = pd.read_sql_query(f'SELECT * FROM ennemy_1_machine_state WHERE id_rec = {RECORD_ID}', conn)
conn.close()

# Preprocess

## Split column [v]

In [4]:
def split_column_v(df_0: pd.DataFrame) -> pd.DataFrame: 
    '''
    splits column [v] to its parts.
    '''

    df_1 = df_0\
        .drop(columns=['v'])
    
    df_2 = df_0\
        .loc[:, 'v']\
        .str\
        .split(':', expand=True)\
        .set_axis(COLUMN_NAMES, axis=1)
    
    df_3 = pd.concat([df_1, df_2], axis=1)

    return df_3

df_our = split_column_v(df_our)
df_bandit = split_column_v(df_bandit)

## Split transformation matrix

In [5]:
def split_transform_matrix(df_0: pd.DataFrame) -> pd.DataFrame:
    '''
    splits transform matrix and converts to float.
    '''

    df_1 = df_0\
        .drop(columns=['transform_matrix'])\
        .copy()
    
    df_2 = df_0\
        .loc[:, 'transform_matrix']\
        .str\
        .split(';', expand=True)\
        .astype(float)\
        .add_prefix('transform_matrix_')
    
    df_3 = pd.concat([df_1, df_2], axis=1)
    
    return df_3

df_our = split_transform_matrix(df_our)
df_bandit = split_transform_matrix(df_bandit)

## Split velocity vector

In [6]:
def split_velocity_vector(df_0: pd.DataFrame) -> pd.DataFrame:
    '''
    splits transform matrix and converts to float.
    '''

    COLS = ['velocity_x', 'velocity_y', 'velocity_z']

    df_1 = df_0\
        .drop(columns=['velocity'])\
        .copy()
    
    df_2 = df_0\
        .loc[:, 'velocity']\
        .str\
        .split(';', expand=True)\
        .astype(float)\
        .set_axis(COLS, axis=1)
    
    df_3 = pd.concat([df_1, df_2], axis=1)
    
    return df_3

df_our = split_velocity_vector(df_our)
df_bandit = split_velocity_vector(df_bandit)

## Build target identified flag

In [7]:
df_our.loc[:, 'bandit_is_targetted'] = df_our.loc[:, 'target_name'].isna().astype(int)
df_bandit.loc[:, 'bandit_is_targetted'] = df_bandit.loc[:, 'target_name'].isna().astype(int)

df_our.drop(columns=['target_name'], inplace=True)
df_bandit.drop(columns=['target_name'], inplace=True)

## Remove unncessary columns

In [8]:
COLS = ['wreck_flag', 'landed_level']
df_our.drop(columns=COLS, errors='ignore', inplace=True)
df_bandit.drop(columns=COLS, errors='ignore', inplace=True)

## Convert fields to float

In [9]:
COLS = ['health_level', 'brake_level', 'flaps_level']
for col in COLS:
    df_our[col] = df_our\
        .loc[:, col]\
        .astype(np.float64)

    df_bandit[col] = df_bandit\
        .loc[:, col]\
        .astype(np.float64)

## Concat aircraft data

In [10]:
COLS_BANDIT_NOT_HAVE = ['health_level', 'brake_level', 'flaps_level', 'bandit_is_targetted']

df_0 = df_our.copy()
df_0.loc[:, 'aircraft_id'] = 'ally_1'

df_1 = df_bandit\
    .drop(columns=COLS_BANDIT_NOT_HAVE)\
    .copy()

df_2 = df_0.merge(
    right=df_1,
    left_on=['id_rec', 'c'],
    right_on=['id_rec', 'c'],
    how='inner',
    suffixes=(None, '_bandit'))

if len(df_2) != len(df_0): 
    raise ValueError('there is insyncronoisation between our craft and bandit')

df_3 = df_bandit.copy()
df_3.loc[:, 'aircraft_id'] = 'ennemy_1'

df_4 = df_our\
    .drop(columns=COLS_BANDIT_NOT_HAVE)\
    .copy()

df_5 = df_3.merge(
    right=df_4,
    left_on=['id_rec', 'c'],
    right_on=['id_rec', 'c'],
    how='inner',
    suffixes=(None, '_bandit'))

df_6 = pd.concat([df_2, df_5], ignore_index=True)

preprocessed_data = df_6\
    .set_index(['aircraft_id', 'id_rec', 'c'])\
    .sort_index()\
    .reset_index()\
    .copy()

del df_0, df_1, df_2, df_3, df_4, df_5, df_6

## Exclude timesteps with no health

In [11]:
df_0 = preprocessed_data\
    .query('health_level == 0')\
    .loc[:, ['id_rec', 'c']]\
    .groupby('id_rec')\
    .min()

for id_rec, srs in df_0.iterrows():
    c = srs['c']
    ix_to_drop = preprocessed_data\
        .query('id_rec == @id_rec')\
        .query('c >= @c')\
        .index

    preprocessed_data = preprocessed_data\
        .drop(ix_to_drop, axis=0)\
        .reset_index(drop=True)

del df_0

## Rename field

In [12]:
preprocessed_data.rename(
    columns={'c': 'timestep', 'id_rec': 'record_id'},
    inplace=True)

# Save

In [13]:
preprocessed_data.to_csv(SAVE_DIR, index=False, sep=';')